post processing of hhblits

In [1]:
import pickle as pkl
from typing import List,Dict,Tuple,Union
from pathlib import Path
import pandas as pd
search_result:List[Tuple[Path,List[Tuple[Path,List[str]]]]]=pkl.load(open('blit_out.pkl','rb'))

In [2]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [3]:
def proindice_to_genindice(genome:Union[str,Seq],prob:int,proe:int,transannot:str)->Tuple[int,int]:
    frame,direction=transannot
    frame=int(frame)
    b_,e_=frame+prob*3,frame+proe*3
    if direction=='s':
        return b_,e_
    else:
        return len(genome)-e_, len(genome)-b_

def genindice_to_proindice(genome:Union[str,Seq],genb:int,gene:int,transannot:str)->Tuple[int,int]:
    frame,direction=transannot
    frame=int(frame)
    
    if direction=='s':
        b_,e_=(genb-frame)//3,(gene-frame)//3
        return b_,e_
    else:
        return (len(genome)-frame-gene)//3,(len(genome)-frame-genb)//3
        

def genome_annot_indice(genome:Union[str,Seq],genb:int,gene:int,transannot:str)->Tuple[int,int]:
    'return: genseq segments '
    frame,direction=transannot
    # frame=int(frame)
    o=genome[genb:gene]
    o=o if direction=='s' else o[::-1]
    return o

In [4]:
from dna_features_viewer import GraphicFeature, GraphicRecord
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.patches as patches

In [5]:

from collections import namedtuple
from matplotlib import colormaps
import matplotlib.colors as mcolors

Hit=namedtuple('Hit',('domain_accession','domain_annotation',
    'prob','eval','pval','score','ss','begin','end','hmmbegin','hmmend','hmmlength'))
SegMeta=namedtuple('SegMeta',("transannot",'seg_b','seg_e'))

def split_hit_line(hit_line:str)->List[str]:
    i=hit_line
    assert len(i)>94
    return [i[:4],i[4:34],i[34:40],i[40:48],i[48:56],i[56:63],
            i[63:69],i[69:74],i[74:85],i[85:94],i[94:]]

def parse_splited_hit_line(hit:List[str])->Hit:
    domain_accession,*_=hit[1].split(';')
    domain_accession=domain_accession.split('.')[0].strip()
    if len(_)>0:
        domain_annotation=_[0].strip()
    else:
        domain_annotation=domain_accession
        
    prob=float(hit[2].strip())
    e=float(hit[3].strip())
    p=float(hit[4].strip())
    score=float(hit[5].strip())
    ss=float(hit[6].strip())
    begin,end=[int(i.strip())-1 for i in hit[8].split('-')]
    hmmbegin,hmmend=[int(i.strip())-1 for i in hit[9].split('-')]
    hmmlength=int(hit[10].strip().strip('(').strip(')'))
    h=Hit(domain_accession=domain_accession,domain_annotation=domain_annotation,
        prob=prob,eval=e,pval=p,score=score,ss=ss,
        begin=begin,end=end,hmmbegin=hmmbegin,hmmend=hmmend,hmmlength=hmmlength)
    return h



In [6]:
from itertools import accumulate
def get_color(genome_position:float,prob:float):
    ''' domain colors based on relative position'''
    cm=mcolors.LinearSegmentedColormap.from_list('tmp',[(1, 1, 1), colormaps['rainbow'](genome_position)], N=512)
    return cm(prob/100)

def get_color_dict(clan_annotation:pd.Series,used_cm=['tab10','Set2','Accent','Set3','Pastel2','Dark2']):
    ''' clan colors '''
    uni_annotation=clan_annotation.unique()
    lengths=[colormaps[i].N for i in used_cm]
    assert len(uni_annotation)<=sum(lengths),(f'no enough colors:'
         f'{len(uni_annotation)} vs {sum(lengths)}')
    out_dict={}
    cpkt=[0]+list(accumulate(lengths))
    cm_pointer=0
    for i,j in enumerate(uni_annotation):
        if i>=cpkt[cm_pointer+1]:
            cm_pointer+=1
        color=colormaps[used_cm[cm_pointer]](i-cpkt[cm_pointer])
        out_dict[j]=color
    return out_dict

In [8]:
# from matplotlib.backends.backend_pdf import PdfPages
# if 0:
#     with PdfPages('poc2.pdf') as pdf:
#         for genome_res in search_result[:2]:
#             genome_res:Tuple[Path,List[Tuple[Path,List[str]]]]
#             genome,o=parse_genome_res(genome_res)
#             fig,ax=plt.subplots(1,1,sharex=True,figsize=(30,10))
#             features=[]
#             len_genome=len(genome)
#             for (transannot,prob,proe),hit in o:
#                 hit:Hit
                
#                 genomeb,genomee=proindice_to_genindice(genome,prob+hit.begin,prob+hit.end,transannot)
#                 strand=1 if transannot.endswith('s') else -1
#                 rele_pos=(genomeb+genomee)/2/len_genome
#                 features.append(GraphicFeature(start=genomeb, end=genomee, strand=strand,
#                                 label=hit.domain_annotation,color=get_color(rele_pos,hit.prob)))
                
#             record = GraphicRecord(sequence_length=len_genome, features=features)
#             record.plot(ax=ax,figure_width=20)
#             fig.suptitle(genome.id)
#             pdf.savefig(fig)
#             plt.close(fig)

involve clans

In [7]:
import numpy as np
from functools import partial

def df_to_dict(df:pd.DataFrame,k:str,v:str):
    return df.set_index(k)[v].T.to_dict()
clans=pd.read_csv('Pfam-A.clans.tsv',delimiter='\t',header=None)
clans.columns=['domain','clan','clan_name','domain_name','domain_annotation']
clan_dict=df_to_dict(clans,'domain','clan')
clan_name_dict=df_to_dict(clans,'domain','clan_name')

def query_clan(domain_accession:str,domain_annotation):
    clan_accession=clan_dict.get(domain_accession,np.nan)
    if clan_accession!=clan_accession:
        clan_accession=domain_accession
    clan_annotation=clan_name_dict.get(domain_accession,np.nan)
    if clan_annotation!=clan_annotation:
        clan_annotation=domain_annotation
    return clan_accession,clan_annotation



In [8]:
from matplotlib.backends.backend_pdf import PdfPages
# genome,o=parse_genome_res(genome_res)
def parse_genome_res(genome_res:Tuple[Path,List[Tuple[Path,List[str]]]]):
    stem=genome_res[0].stem
    stems=stem.replace(':segs','').split('|')
    genome_fasta_path=genome_res[0].with_stem(stem.replace('segs','genome'))
    genome:SeqRecord=SeqIO.read(genome_fasta_path,'fasta')
    o:List[Tuple[SegMeta,Hit]]=[]
    
    for seg_res in genome_res[1]:
        if len(seg_res[1])>0:
            _,transannot,prob,proe=seg_res[0].stem.split('#')
            prob,proe=int(prob),int(proe)
            segmeta=SegMeta(transannot=transannot,seg_b=prob,seg_e=proe)
            for h in seg_res[1]:
                hit=parse_splited_hit_line(h)
                o.append((segmeta,hit))   
    return genome,o
def genome_res_to_hits_df(genome:SeqRecord,parse_o:List[Tuple[SegMeta,Hit]])->pd.DataFrame:
    hits_df=pd.concat([pd.DataFrame([i[1] for i in parse_o]),pd.DataFrame([i[0] for i in parse_o])],axis=1) 
    hits_df['gen_b'],hits_df['gen_e']=np.vectorize(partial(
        proindice_to_genindice,genome=genome))(
        prob=hits_df['begin']+hits_df['seg_b'],
        proe=hits_df['end']+hits_df['seg_b'],
        transannot=hits_df['transannot'])

    hits_df['gen_hmm_b'],hits_df['gen_hmm_e']=np.vectorize(partial(
        proindice_to_genindice,genome=genome))(
        prob=hits_df['begin']-hits_df['hmmbegin']+1+hits_df['seg_b'],
        proe=hits_df['end']+hits_df['hmmlength']-hits_df['hmmend']+hits_df['seg_b'],
        transannot=hits_df['transannot'])
    hits_df['clan_accession'],hits_df['clan_annotation']=np.vectorize(query_clan)(
        hits_df['domain_accession'],hits_df['domain_annotation'])
    return hits_df


def inspect_genome_res(genome_res:Tuple[Path,List[Tuple[Path,List[str]]]]):
    genome,o=parse_genome_res(genome_res)
    hits_df=genome_res_to_hits_df(genome,o)
    genome_id=genome.id
    count=0
    len_genome=len(genome)
    color_dict=get_color_dict(hits_df['clan_annotation'])
    with PdfPages(f'{genome_id}.pdf') as pdf:
        while count<4:
            fig,ax=plt.subplots(1,1,sharex=True,figsize=(30,12))
            features=[]
            titles=[f'{genome_id}-origin\n50prob-domaincolor-hitindice',f'{genome_id}-filterdomain\n90prob-domaincolor-hitindice',
                    f'{genome_id}-color-as-clan\n90prob-clancolor-hitindice',f'{genome_id}-cover-whole-hmm\n90prob-clancolor-hmmindice']
            for _,s in hits_df.iterrows():
                strand=1 if s['transannot'].endswith('s') else -1
                if count==0:
                    if s['prob']>=0:
                        genomeb,genomee=s['gen_b'],s['gen_e']
                        rele_pos=(genomeb+genomee)/2/len_genome
                        features.append(GraphicFeature(start=genomeb, end=genomee, strand=strand,
                                        label=s['domain_annotation'],color=get_color(rele_pos,s['prob'])))
                elif count==1:
                    if s['prob']>=90:
                        genomeb,genomee=s['gen_b'],s['gen_e']
                        rele_pos=(genomeb+genomee)/2/len_genome
                        features.append(GraphicFeature(start=genomeb, end=genomee, strand=strand,
                                        label=s['domain_annotation'],color=get_color(rele_pos,s['prob'])))
                elif count==2:
                    if s['prob']>=90:
                        genomeb,genomee=s['gen_b'],s['gen_e']
                        features.append(GraphicFeature(start=genomeb, end=genomee, strand=strand,
                                        label=s['clan_annotation'],color=color_dict[s['clan_annotation']]))
                elif count==3:
                    if s['prob']>=90:
                        genomeb,genomee=s['gen_hmm_b'],s['gen_hmm_e']
                        features.append(GraphicFeature(start=genomeb, end=genomee, strand=strand,
                                        label=s['clan_annotation'],color=color_dict[s['clan_annotation']]))
                    
            record = GraphicRecord(sequence_length=len_genome, features=features)
            record.plot(ax=ax,figure_width=20)
            # fig.show()
            fig.suptitle(titles[count],fontsize=20)
            pdf.savefig(fig)
            plt.close(fig)
            count=count+1

In [13]:
# from matplotlib import rcParams
# type(rcParams['axes.prop_cycle'])

In [9]:
def all_in_one_parse(genome_res:Tuple[Path,List[Tuple[Path,List[str]]]]):
    genome,parse_o=parse_genome_res(genome_res)
    hits_df=genome_res_to_hits_df(genome,parse_o)
    return genome,parse_o,hits_df

genome_res = search_result[1]
genome,parse_o,hits_df=all_in_one_parse(genome_res)
# genome,o=parse_genome_res(genome_res)
# hits_df=genome_res_to_hits_df(genome,o)

# inspect_genome_res(genome_res)

In [10]:
search_result_dict = {i[0].stem.strip(':segs'):i for i in search_result}

In [11]:
zika_res=search_result_dict['ZIKV||AY632535']
zika_os=all_in_one_parse(zika_res)


In [35]:
for i in ['ZIKV||AY632535','EBOV||AF086833',
    'MeV||AB016162','SARS-CoV-2||MN908947',
    'RPbV||KC815311','TBEV-FE||X07755',
    'WPSSPV||MG600057','KNDV||MW093492']:
    try:
        genome_res=search_result_dict[i]
        inspect_genome_res(genome_res)
    except:
        print(i)

/home/hugheslab1/zfdeng/.local/lib/python3.12/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in query_clan (vectorized)
  outputs = ufunc(*inputs)
/home/hugheslab1/zfdeng/.local/lib/python3.12/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in query_clan (vectorized)
  outputs = ufunc(*inputs)
/home/hugheslab1/zfdeng/.local/lib/python3.12/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in query_clan (vectorized)
  outputs = ufunc(*inputs)
/home/hugheslab1/zfdeng/.local/lib/python3.12/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in query_clan (vectorized)
  outputs = ufunc(*inputs)
/home/hugheslab1/zfdeng/.local/lib/python3.12/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in query_clan (vectorized)
  outputs = ufunc(*inputs)


WPSSPV||MG60005


/home/hugheslab1/zfdeng/.local/lib/python3.12/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: invalid value encountered in query_clan (vectorized)
  outputs = ufunc(*inputs)


In [39]:
genome_res=search_result_dict['OSV-1||MF324848']
inspect_genome_res(genome_res)

'ZIKV||AY632535'
    hint: too brutal to use hmm indice directly
    cause pseudo overlap in Orbi_VP1,P-loop_NTPase
    some families are actually multidomain (e.g. RdRP3); partial hit is ONLY partial hit.

'EBOV||AF086833'
    pseudo double hit of Filo_glycop
    G-7-MTase & NADP_Rossmann(e.g Fstj): mighe be the same thing samething


'MeV||AB016162'
    pseudo double hit of Paramyxovirin_C

'SARS-CoV-2||MN908947'
    so many hits, disaster
    to be analysed again after hhm/hhm-align and redefine clan
    

'RPbV||KC815311':
    homology of zika, classic false-negative with PF00998 instead of zika's rdrp (PF00792) hit
    one more hit found (1st on the left)

'TBEV-FE||X07755'
    homology of zika, out-of-distribution
    very tidy, perfect

'WPSSPV||MG600057'
    homology of measles, with long segs of false-negative (no hit in Interproscan)
    no more domain hit

'KNDV||MW093492
    homology of ebola, with long segs of false-negative (no hit in Interproscan)
    some DUFs and ZFs hit in that missing area, with low(50-90) prob

'OSV-1||MF324848'
    homology of covid, random chosen.

Legend for each cases:
1. Standard refs for Covid/Zika/Ebola/Measles
    ZIKV||AY632535

    EBOV||AF086833

    MeV||AB016162

    SARS-CoV-2||MN908947

2. Representative homologs:
    RPbV||KC815311: 
        homology of zika, classic false-negative with PF00998 instead of zika's rdrp (PF00792) hit

    TBEV-FE||X07755:
        homology of zika, out-of-distribution orphan

    WPSSPV||MG600057
        homology of measles, with long segs of false-negative (no hit in Interproscan)

    KNDV||MW093492
        homology of ebola, with long segs of false-negative (no hit in Interproscan)

In [13]:
# from glob import glob
for i in Path('.').iterdir():
    if i.suffix=='.pdf':
        new_stem=i.stem.replace('|','#')
        i.rename(i.with_stem(new_stem))
    
 

Progress:
hh-blits returns every possible hits as we expected

hh-blits could hit domains ignored by Interproscan (RPbV||KC815311,OSV-1||MF324848)

hh-blits could fix false loops (multi-hit. e.g MeV||AB016162)

Issues:
hh-blits is slower than interproscan (avg: 20core * 20s; interproscan: 4core * 15s) 
    reason: interproscan could predict ORFs, 
            while translation of multiple segs (most of them are useless) will be analyzed tandemly in hh-blits

False ORF could yield false hit (small fragments) with high probability

Pfam clan is very rough. rerun with hhblits is definitely needed.

Too brutal to use hmm indice directly, which might cause pseudo overlap

hitting of domains ignored by Interproscan is still not perfect (WPSSPV||MG600057, KNDV||MW093492)

SARS-CoV-2||MN908947: too many hits to be disaster


In [26]:
zika_os[2]

,domain_accession,domain_annotation,prob,eval,pval,score,ss,begin,end,hmmbegin,...,hmmlength,transannot,seg_b,seg_e,gen_b,gen_e,gen_hmm_b,gen_hmm_e,clan_accession,clan_annotation
0,PF19215,CoV_NSP15_C,100.0,3.700000e-36,7.200000e-40,303.1,0.0,1174,1294,1,...,155,0s,2069,3485,9729,10089,9729,10092,CL0695,EndoU
1,PF17977,zf-RING_13,99.7,1.800000e-22,3.300000e-26,174.0,0.0,636,696,1,...,68,0s,2069,3485,8115,8295,8115,8304,CL0229,RING
2,PF00978,RdRP_2,99.6,1.900000e-21,4.400000e-25,218.1,0.0,331,571,148,...,440,0s,2069,3485,7200,7920,6759,8109,CL0027,RdRP
3,PF00998,RdRP_3,99.5,6.300000e-19,1.500000e-22,197.8,0.0,331,574,135,...,479,0s,2069,3485,7200,7929,6798,8238,CL0027,RdRP
4,PF17873,Rep_1B,99.4,4.700000e-18,8.600000e-22,140.5,0.0,709,760,1,...,53,0s,2069,3485,8334,8487,8334,8490,CL0105,Hybrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,PF12046,CCB1,74.1,1.300000e-01,2.800000e-05,35.7,0.0,86,132,62,...,171,2s,792,977,2636,2774,2453,2948,PF12046,CCB1
98,PF02340,PRRSV_Env,77.3,9.200000e-02,2.000000e-05,40.3,0.0,211,286,139,...,233,2s,3492,3792,11111,11336,10697,11384,PF02340,PRRSV_Env
99,PF00951,Arteri_Gl,100.0,1.800000e-65,3.500000e-69,432.9,0.0,77,244,6,...,173,2s,4012,4257,12269,12770,12254,12773,PF00951,Arteri_Gl
100,PF01481,Arteri_nucleo,100.0,5.800000e-46,1.100000e-49,271.1,0.0,19,123,10,...,115,2s,4423,4549,13328,13640,13301,13643,PF01481,Arteri_nucleo


In [42]:
[i[0].stem for i in search_result]

['AAbV||GBBW01007738:segs',
 'OSV-1||MF324848:segs',
 'EAV||X53459:segs',
 'APRAV||KP026921:segs',
 'SHFV||AF180391:segs',
 'SHEV||KM677927:segs',
 'FSVV||KR862307:segs',
 'ZMbV-1||KT166441:segs',
 'KRCV-2||KC787658:segs',
 'DeBMAV||KP126831:segs',
 'KRTGV-1||JX473849:segs',
 'PBJV||KR139839:segs',
 'KKCBV||KT447550:segs',
 'MYBV-1||KM110938:segs',
 'KRCV-1||KC787630:segs',
 'PRRSV-2||U87392:segs',
 'RtMrufAV||KP280006:segs',
 'RtClonAV||KU302440:segs',
 'RtEiAV||KY369968:segs',
 'PRRSV-1||M96262:segs',
 'RtMcAV||KY369967:segs',
 'LDV||U15146:segs',
 'RtClanAV||KY369969:segs',
 'WPDV||JN116253:segs',
 'TSHSV-NX1||MH447987:segs',
 'CBHPTAV||MG600025:segs',
 'GGGSAV||MG600023:segs',
 'HOFAV||MG600022:segs',
 'MLeV||GECV01031551:segs',
 'BtCoV-AMA-L-F||MT663548:segs',
 'BtCoV CDPHE15||KF430219:segs',
 'HipPBCoV-CHB25||MN611525:segs',
 'ACoV-WA3607||MK472070:segs',
 'BtCoV HKU10||JQ989270:segs',
 'BtRf-AlphaCoV||KJ473807:segs',
 'HCoV_229E||AF304460:segs',
 'HCoV_229E||JX503061:segs',
 'LR

## note for OSV-1||MF324848-origin
### case1. the clan nearby should include this two entry.
1. Orbi_VP1:
    https://www.ebi.ac.uk/interpro/entry/pfam/PF05788/
    Also RdRp

2. IFR3_antag:
    https://www.ebi.ac.uk/interpro/entry/pfam/PF14754/
    Also proteinase, and hit prob=100% here.

### case2 accessory domains of nearby function domain
1. Hybird (Rep_1B)
    https://www.ebi.ac.uk/interpro/entry/pfam/PF17873/
    "a regulatory domain found in replicase polyprotein 1ab found in Arterivirus"

    hybrid:
    This superfamily contains proteins with a hybrid motif embedded in structurally diverse proteins.
    PMID: 8081735

2. DUF3388
    https://www.ebi.ac.uk/interpro/entry/pfam/PF11868/
    'functionally uncharacterised. This protein is found in bacteria'
    'associated with Pfam:PF01842: ACT domain'

3. RING (zf-RING_13)
    https://www.ebi.ac.uk/interpro/entry/pfam/PF17977/
    RING/Ubox like zinc-binding domain

### case3 valid multiple hits
1. Arteri_Gl
    https://www.ebi.ac.uk/interpro/entry/pfam/PF00951/
    Gl envelope protein encoded by Arteriviruses 

### case4 multiple hits should be integrated
1. Peptidase_S46
    https://www.ebi.ac.uk/interpro/entry/pfam/PF10459/
    member of Peptidase_PA

In [139]:
hits_df['domain_annotation'].value_counts()[hits_df['domain_annotation'].value_counts()>1]

domain_annotation
Arteri_Gl        2
Peptidase_S46    2
Name: count, dtype: int64

In [35]:
hits_df[hits_df['domain_annotation']=='Peptidase_S46'].T

,89,94
domain_accession,PF10459,PF10459
domain_annotation,Peptidase_S46,Peptidase_S46
prob,95.2,74.0
eval,0.00011,0.12
pval,0.0,0.000028
score,84.0,57.3
ss,0.0,0.0
begin,1393,1487
end,1418,1513
hmmbegin,43,640


In [39]:
hits_df[hits_df['clan_annotation']=='Arteri_Gl'].T

,96,99
domain_accession,PF00951,PF00951
domain_annotation,Arteri_Gl,Arteri_Gl
prob,100.0,100.0
eval,0.0,0.0
pval,0.0,0.0
score,344.7,432.9
ss,0.0,0.0
begin,17,77
end,182,244
hmmbegin,18,6
